In [7]:
from PIL import Image
import numpy as np

### Gradient Calculation & Non Maximum Supression

In [15]:
# Function to convert RGB image to grayscale
def rgb_to_gray(rgb_image):
    red, green, blue = rgb_image[:,:,0], rgb_image[:,:,1], rgb_image[:,:,2]
    gray = 0.2989 * red + 0.5870 * green + 0.1140 * blue
    return gray

# Function to perform grayscale convolution
def convolve_grayscale(image, kernel):
    output = np.zeros_like(image)
    image_padded = np.zeros((image.shape[0]+kernel.shape[0]-1,image.shape[1] + kernel.shape[1]-1))
    image_padded[kernel.shape[0]-2:-1:,kernel.shape[1]-2:-1:] = image
    image_padded[0,0] = image[0,0]
    image_padded[-1,-1] = image[-1,-1]
    for x in range(image.shape[1]):
        for y in range(image.shape[0]):
            output[y, x] = (kernel * image_padded[y: y+kernel.shape[0], x: x+kernel.shape[1]]).sum()
    return output


# Function for non-maximum suppression
def non_max_suppression(img, directions):
    M, N = img.shape
    result = np.zeros((M, N), dtype=np.int32)
    angle = directions * 180. / np.pi
    angle[angle < 0] += 180
    
    for i in range(1, M-1):
        for j in range(1, N-1):
            try:
                q = 255
                r = 255
                
                if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                    q = img[i, j+1]
                    r = img[i, j-1]
                elif (22.5 <= angle[i, j] < 67.5):
                    q = img[i+1, j-1]
                    r = img[i-1, j+1]
                elif (67.5 <= angle[i, j] < 112.5):
                    q = img[i+1, j]
                    r = img[i-1, j]
                elif (112.5 <= angle[i, j] < 157.5):
                    q = img[i-1, j-1]
                    r = img[i+1, j+1]

                if (img[i, j] >= q) and (img[i, j] >= r):
                    result[i, j] = img[i, j]
                else:
                    result[i, j] = 0

            except IndexError as e:
                pass
    
    return result

### Double Threshold

In [16]:
# Function for double thresholding
def threshold(img, weak, strong, low_threshold_ratio=0.05, high_threshold_ratio=0.09):
    
    high_threshold = img.max() * high_threshold_ratio
    low_threshold = high_threshold * low_threshold_ratio
    
    M, N = img.shape
    res = np.zeros((M, N), dtype=np.int32)
    
    weak = np.int32(weak)
    strong = np.int32(strong)
    
    strong_i, strong_j = np.where(img >= high_threshold)
    zeros_i, zeros_j = np.where(img < low_threshold)
    
    weak_i, weak_j = np.where((img <= high_threshold) & (img >= low_threshold))
    
    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak
    
    return res

### Edge Tracking by Hysteresis

In [17]:
# Function for hysteresis
def hysteresis(img, weak, strong):
    M, N = img.shape  
    for i in range(1, M-1):
        for j in range(1, N-1):
            if (img[i, j] == weak):
                try:
                    if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                        or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                        or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                        img[i, j] = strong
                    else:
                        img[i, j] = 0
                except IndexError as e:
                    pass
    return img

In [20]:
# Gaussian blur kernel
gaussian_blur = np.array([[1,  4,  6,  4, 1],
                          [4, 16, 24, 16, 4],
                          [6, 24, 36, 24, 6],
                          [4, 16, 24, 16, 4],
                          [1,  4,  6,  4, 1]]) / 256

# Sobel filters for x and y directions
x_direction_kernel = np.array([[-1, 0, 1],
                               [-2, 0, 2],
                               [-1, 0, 1]], np.float32)

y_direction_kernel = np.array([[-1, -2, -1],
                               [ 0,  0,  0],
                               [ 1,  2,  1]], np.float32)

# Load and convert the image to grayscale
image_path = 'Dog.png'
img_rgb = np.array(Image.open(image_path))
img_gray = rgb_to_gray(img_rgb)

# Noise Reduction using Gaussian Blur
img_blurred = convolve_grayscale(img_gray, gaussian_blur)

# Gradient Calculation using Sobel Filters
x_intensity = convolve_grayscale(img_blurred, x_direction_kernel)
y_intensity = convolve_grayscale(img_blurred, y_direction_kernel)
gradient_magnitude = np.hypot(x_intensity, y_intensity)
gradient_magnitude = gradient_magnitude / gradient_magnitude.max() * 255
gradient_direction = np.arctan2(y_intensity, x_intensity)

# Non-maximum suppression
img_suppressed = non_max_suppression(gradient_magnitude, gradient_direction)

# Double Threshold
weak_threshold = 75
strong_threshold = 255
img_thresholded = threshold(img_suppressed, weak_threshold, strong_threshold, 0.05, 0.15)

# Applying Hysteresis
img_final = hysteresis(img_thresholded, weak_threshold, strong_threshold)

# Save the result as an image
result_image = Image.fromarray(img_final).convert('RGB')
result_image.save('canny_image.jpg')